# EMLOpt - Google Colab notebook


## Dependencies

NOTE: Restart the runtime may be required

In [ ]:
from google.colab import files
files.upload()
!unzip eml-opt.zip

!pip install tensorflow-addons
!pip install tensorflow-probability
!pip install protobuf==3.19.0
!pip install wandb

!mkdir -p dependencies/cplex
!wget https://api.wandb.ai/artifactsV2/gcp-us/veri/QXJ0aWZhY3Q6MjU1ODAzMjI=/69b1b89a73a7d0931fbfdb355eb147c3 -O dependencies/cplex/cplex_studio1210.linux-x86-64.bin
!wget https://api.wandb.ai/artifactsV2/gcp-us/veri/QXJ0aWZhY3Q6MjU1ODAzMjI=/97133b747b0114a4e3dba77ab26d68d5 -O dependencies/cplex/response.properties

!pip install cplex==12.9.0.0
!pip install docloud==1.0.375
!pip install docplex==2.10.155
!pip install ortools==9.1.9490

!sh dependencies/cplex/cplex_studio1210.linux-x86-64.bin -f response.properties
!python3 /opt/ibm/ILOG/CPLEX_Studio1210/python/setup.py install


## Optimization

In [ ]:
!pip install tensorflow-model-optimization

import sys
sys.path.append("./eml-opt") 

import numpy as np
from emlopt import surrogates, solvers
from emlopt.search_loop import SearchLoop
from emlopt.problem import build_problem
from emlopt.wandb import WandbContext
from experiments.problems.quantization.tpc import build_tpc, constraint_max_bits

tpc_obj = build_tpc()

CONFIG = {
    "verbosity": 2,

    "iterations": 100,
    "starting_points": 3,

    "surrogate_model": {
        "type": "stop_ci",
        "epochs": 999,
        "learning_rate": 5e-3,
        "weight_decay": 1e-4,
        "batch_size": None,
        "depth": 2,
        "width": 30,
        "ci_threshold": 5e-2,
    },

    "milp_model": {
        "type": "simple_dist",
        "backend": "cplex",
        "bound_propagation": "both",
        "lambda_ucb": 1,
        "solver_timeout": 120,
    }
}

problem = build_problem("tpc", tpc_obj, ["int"]*41, [[2, 8]]*41, constraint_max_bits)
search = SearchLoop(problem, CONFIG)

search.known_points = (np.expand_dims(np.array([4]*41), 0), -0.755)

wandb_cfg = WandbContext.get_defatult_cfg()
wandb_cfg['experiment_name'] = "TPC"

with WandbContext(wandb_cfg, search):
    search.run()